In [10]:
import tensorflow as tf
import pathlib

In [11]:
data_url = ''

In [12]:
zip_dir = tf.keras.utils.get_file('fruits360.zip', origin=data_url, extract=True)

In [6]:
path = pathlib.Path(zip_dir).parent / 'fruits360_filtered'

In [7]:
train_dir = path / 'Training'
validation_dir = path / 'Test'

In [8]:
total_train = 11398
total_val = 4752

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [11]:
train_img_gen = ImageDataGenerator(
    rescale=1./255, rotation_range=40, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
    horizontal_flip=True, fill_mode='nearest'
)

In [12]:
validation_img_gen = ImageDataGenerator(rescale=1./255) 

In [13]:
batch_size = 16
img_height = 100
img_width = 100
channel = 3

In [15]:
train_data_gen = train_img_gen.flow_from_directory(
    batch_size=batch_size, directory=train_dir, target_size=(
        img_height, img_width
    )
)

Found 11398 images belonging to 120 classes.


In [16]:
val_data_gen = validation_img_gen.flow_from_directory(
    batch_size=batch_size, directory=validation_dir, target_size=(
        img_height, img_width
    )
)

Found 4752 images belonging to 120 classes.


In [20]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers

In [21]:
base_model = VGG16(
    input_shape=(
        img_height, img_width, channel
    ), weights='imagenet', include_top=False
)

58892288/58889256 [==============================] - 20s 0us/step


In [22]:
base_model.trainable = False
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100, 100, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 100, 100, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 100, 100, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 50, 50, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 50, 50, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 50, 50, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 25, 25, 128)       0     

In [23]:
model = tf.keras.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(1000, activation='relu'),
    layers.Dense(120, activation='softmax')
])

In [24]:
optimizer = tf.keras.optimizers.Adam(0.001)

In [26]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy']
)

In [27]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 3, 3, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 4608)              0         
_________________________________________________________________
dense (Dense)                (None, 1000)              4609000   
_________________________________________________________________
dense_1 (Dense)              (None, 120)               120120    
Total params: 19,443,808
Trainable params: 4,729,120
Non-trainable params: 14,714,688
_________________________________________________________________


In [29]:
model.fit_generator(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=5,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size
)

C:\Users\hp\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
712/712 [==============================] - 55s 68ms/step - loss: 1.9980 - accuracy: 0.5130 - val_loss: 1.0321 - val_accuracy: 0.7062
Epoch 2/5
712/712 [==============================] - 42s 59ms/step - loss: 0.6461 - accuracy: 0.8132 - val_loss: 0.6388 - val_accuracy: 0.8239
Epoch 3/5
712/712 [==============================] - 39s 55ms/step - loss: 0.4325 - accuracy: 0.8694 - val_loss: 0.4935 - val_accuracy: 0.8415
Epoch 4/5
712/712 [==============================] - 40s 57ms/step - loss: 0.3459 - accuracy: 0.8920 - val_loss: 0.6369 - val_accuracy: 0.8152
Epoch 5/5
712/712 [==============================] - 38s 54ms/step - loss: 0.2834 - accuracy: 0.9115 - val_loss: 0.4978 - val_accuracy: 0.8508
